# Microsoft malware classification
_Klasifikacija zlonamjernog softvera_

PRIRODOSLOVNO MATEMATIČKI FAKULTET - MATEMATIČKI ODSJEK<br>
Projektni prijedlog iz kolegija Strojno učenje <br>
__Travanj 2020.__ <br>

_Stanišić Matea, Škrabo Petra, Terzanović Mateja, Tolja Margarita_

<br>

Jedna od najfinanciranijih i bitnijih IT industrija današnjice zasigurno je ona koja se bavi _malwareima_. Kako se većina zlonamjernog softvera kreira konstantnim (_malicioznim_) mjenjanjem već postojećih _malwarea_, dolazi na važnost njihovo klasificiranje. 

Ideja za projekt dolazi sa [Kaggle natječaja](https://www.kaggle.com/c/malware-classification/overview) - _Microsoft malware classification_.<br>

U ovoj _Jupiter_ bilježnici prikazati ćemo podatke koje je omogućio natječaj. Orginalni podaci veličine su oko pola TB te su podjeljeni u podatke za treniranje i testiranje. Podaci za treniranje algoritma za klasifikaciju imaju specificiranu klasu familije kojoj pripada svaki od _malwarea_. Moguće su sljedeće familije:
    1. Ramnit
    2. Lollipop
    3. Kelihos_ver3
    4. Vundo
    5. Simba
    6. Tracur
    7. Kelihos_ver1
    8. Obfuscator.ACY
    9. Gatak

Potrebno je, za svaki _malware_ identificiran jedinstvenim _hash_ kodom iz podataka za testiranje, odrediti kojoj od navedenih klasa pripada. <br> 

Podaci za svaki _malware_ sastoje se od dvije datoteke:
 - __.bytes__ koja sadrži heksadecimalnu reprezentaciju binarnog sadržaja _malwarea_
 - __.asm__ koja sadrži metapodatke izvučene iz binarnih podatataka (npr. korištene funkcije i stringovi u asemblerskom kodu)
<br> <br>


Primjer dvije linije iz jedne __.bytes__ datoteke:

<code>00401000 56 8D 44 24 08 50 8B F1 E8 1C 1B 00 00 C7 06 08
00401010 BB 42 00 8B C6 5E C2 04 00 CC CC CC CC CC CC CC</code>

Prvi broj, npr <code> 00401000 </code> označava početak adrese desno napisane heksadecimalne reprezentacije binarnog koda u memoriji. Ostale, desne, heksadecimalne vrijednosti predstavljaju važne informacije o samom _malwareu_, npr. instrukcije u kodu. To se može vidjeti i u primjeru nekoliko linija iz jedne __.asm__ datoteke:

<code>.text:00401016						       ; ---------------------------------------------------------------------------
.text:00401019 CC CC CC	CC CC CC CC					       align 10h
.text:00401020 C7 01 08	BB 42 00					       mov     dword ptr [ecx],	offset off_42BB08
.text:00401026 E9 26 1C	00 00						       jmp     sub_402C51
.text:00401026						       ; ---------------------------------------------------------------------------</code>


<br>
<br>

## Analiza značajki 
_Features analysis_

_KOMENTAR_ 
[//]: # "Po uzoru na https://github.com/PMF-StrojnoUcenje2018/Overflow/blob/master/pocetna_analiza.ipynb"

Kako orginalni podaci nemaju točno određene _feature_ koje bi upotrijebili u algoritmu za klasificiranje, rudarili smo podatke po uzoru na članak: [_Novel Feature Extraction, Selection and Fusion for Effective Malware Family Classification_](https://www.researchgate.net/publication/283986464_Novel_Feature_Extraction_Selection_and_Fusion_for_Effective_Malware_Family_Classification) korišteći njihov gotovi [kod](https://github.com/ManSoSec/Microsoft-Malware-Challenge) koji su objavili, uz male promjene. Kratka analiza i pregled koda može se vidjeti u [feature_extraction.ipynb]. 

_KOMENTAR_
[//]: # "Treba dodati link na ovu bbilježnicu"

###  _featuri_ dobivene iz __.bytes__ datoteke

#### 1 - gram 
_1G_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### (byte) metadata 
_MDB_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### entropija
_ENT_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### slikovna reprezentacija virusa
_IMG_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### duljina stringova
_STR_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

<br>
<br>

### _featuri_ dobiveni iz .asm datoteke

#### (asm) metadata
_MDA_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### simboli
_SYM_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### opcode
_OPC_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### registri
_REG_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### API
_API_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### sekcije
_SEC_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### _data define_
_DP_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

#### ključne riječi
_KEYW_

In [ ]:
# grafikoni i tablice jednom kad se završi kompresija podataka

<br>
<br>

## Odabir najznačajnijih značajki
_Feature fusion_

_KOMENTAR_
[//]: # "Možemo npr ovako https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e"

In [ ]:
# FEATURE IMPORTANCE
# Feature importance is an inbuilt class that comes with Tree Based Classifiers, 
# we will be using Extra Tree Classifier for extracting the top 10 features for the dataset.

import pandas as pd
import numpy as np

data = pd.read_csv("D://Blogs//train.csv") #path to data
X = data.iloc[:,0:20]  #independent columns
y = data.iloc[:,-1]    #target column i.e price range
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()